In [1]:
import re
from bs4 import BeautifulSoup
import collections
import pandas as pd
import numpy as np
soup = BeautifulSoup(open('/Users/thiberio/Pictures/yeah.html').read(), 'html.parser')

In [2]:
html = soup.prettify().split('\n')

div_blocks = []
active_block  = False
for line_num, line in enumerate(html):
    line = line.strip()
    if not line:
        continue
    
    if line == '<div class="EventSummary">':
        active_block  = True
        div_tag_count = 0
        div_start     = line_num
        continue
    
    if not active_block:
        continue
        
    if '<div' in line:
        div_tag_count += 1
    if '</div' in line:
        div_tag_count -= 1
        if not div_tag_count:
            div_end       = line_num+1
            active_block  = False
            div_blocks.append('\n'.join(html[div_start:div_end]))

In [37]:
wins   = []
draws  = []
losses = []
for block in div_blocks:
#    if not re.search('Formato:[\s\S]+?</b>[\s\S]+?Modern', block):
#        continue
    matches = re.findall('''<tr class="MatchHistoryRow">([\s\S]*?)</tr>''', block)
    for match in matches:
        regex = re.search('<div class="TeamOpponent">\n\s+(.+)', match)
        if regex:
            opponent = regex.group(1)
        else:
            opponent = 'unnamed'

        if opponent == '</div>':
            opponent = 'unnamed'
    
        if '(+1)' in match:
            draws.append(opponent)
        elif '(+0)' in match:
            losses.append(opponent)
        else:
            wins.append(opponent)
print(len(wins), len(draws), len(losses))

277 35 299


In [45]:
df = pd.DataFrame(columns='wins losses total ratio'.split())
for opponent, num_wins in collections.Counter(wins).items():
    if opponent == 'unnamed':
        continue
    df.loc[opponent, 'wins'] = num_wins

for opponent, num_losses in collections.Counter(losses).items():
    if opponent == 'unnamed':
        continue
    df.loc[opponent, 'losses'] = num_losses

df.fillna(0, inplace=True)
df.total = df.T.sum()
df.ratio = df.apply(
    lambda x: x.wins/x.losses if x.losses > 0 else 0,
    axis=1
)

In [46]:
df[df.total >= 3].sort_values('total', ascending=False)

,wins,losses,total,ratio
Grant Fishman,4,5,9,0.800000
Diogo Tamashiro,6,2,8,3.000000
Richie Corrado,3,4,7,0.750000
Wellington Carneiro,5,2,7,2.500000
Jorge Klotz,4,2,6,2.000000
Jordan Munson,3,3,6,1.000000
Daniel Silva,1,4,5,0.250000
Zach Vanderzee,2,3,5,0.666667
Igor Inocima,2,2,4,1.000000
Gabriel Miranda,1,3,4,0.333333


In [49]:
df.sum()

wins      188.000000
losses    214.000000
total     402.000000
ratio      59.466667
dtype: float64